## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [69]:
import pandas as pd #импорт библиотек
import numpy as np
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [70]:
df = pd.read_csv('/datasets/data.csv') #прочитаем предоставленный файл с данными
print(df.head(10)) #получение первых  строк таблицы
df.info() #общая информация о данных таблицы

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

### Вывод

Рассмотрим полученную информацию подробнее.

Всего в таблице 12 столбцов, типы данных - float64, int64, object.

Подробно разберем какие в data столбцы и какую информацию они содержат:

children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

Количество значений в столбцах отличается. Это говорит о том, что в данных есть пропущенные значения.

Каждая строка таблицы содержит информацию о семейном положении, трудовом стаже, образовании и прочей информации из статистики о платёжеспособности клиентов банка. Для проверки рабочих гипотез сначала нужно решить следующие проблемы:
1. Определить и заполнить пропущенные занчения
2. Заменить вещественный тип данных на целочисленный
3. Удалить дубликаты
4. Выделить леммы в столбце "цели получения кредита"
5. Категоризировать данные

### Шаг 2. Предобработка данных

### Обработка пропусков

In [71]:
df['days_employed'] = df['days_employed'].fillna(0) 
df['total_income'] = df['total_income'].fillna(0)
df.isnull().sum() #проверим количество пропусков

#заменим все отрицательные значения в столбце трудовой стаж на положительные
for element in df['days_employed']:
    if element < 0:
        new_element = element*-1
        df['days_employed'] = df['days_employed'].replace(element, new_element)

#преобразуем трудовой стаж в дни
def new_days_employed(row):
    old_days_employed = row['days_employed']
    new_days_employed = row['days_employed']/24
    
    if old_days_employed > 21000: #максимальный возраст 75 лет (минус 18) - примерно 21000 дней 
        return  new_days_employed
    return old_days_employed

#замена столбца возраст с учетом полученных данных
df['additional_days_employed'] = df.apply(new_days_employed, axis=1)
df['days_employed'] = df['additional_days_employed']
df = df.drop('additional_days_employed', 1)

### Вывод

Были обнаружены пропуски в столбцах трудовой стаж и ежемесячный доход. Данные в столбцах могли быть пропущены случайно (во время выгрузки) или специально (человеческий фактор/могли просто не заполнить данные поля). Произвелена заменена нулевыми значениями с целью дальнейшей работы с данными столбцами. Так же отрицательные значения трудового стажа были заменены на положительные, а значения, которые превышали допустимые значения (вероятно, в системе они отражались как часы, а не как дни) переведены в дни. Обработка пропусков в столбцах возраст и доход будет произведена после проверки уникальных значений.

### Замена типа данных

In [72]:
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)

### Вывод

Столбцы были трудовой стаж и ежемесячный доход приведены к целочисленым значениям для удобства отображения и расчетов.

### Обработка дубликатов

In [73]:
#обработка дубликатов
print(df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop = True)

#Поиск уникальных значений для каждого из столбцов с категориальными переменными (вывод закомментирован для удобства отображения)
#print(df['children'].unique())
df.loc[df['children'] == -1, 'children'] = 1 #отрицательного кол-ва детей быть не может, поэтому значение изменено на аналогичное положительное
df.loc[df['children'] == 20, 'children'] = 2 #вероятно, здесь допущена опечатка, произведена замена на подходящее по смыслу значение

#print(df['dob_years'].unique())

#обработка нулевых значений в столбце возраст 

def new_age_count(row):
    age = row['dob_years']
    new_days_employed = 18 + row['days_employed']/365
    
    if age == 0:
        if row['days_employed'] > 0:
            return  new_days_employed
    return age

df['additional_age'] = df.apply(new_age_count, axis=1)

df['dob_years'] = df['additional_age']
df = df.drop('additional_age', 1)

#print(df[df['dob_years'] == 0 ].count())

#print(df['education'].unique())

df['education_lowercase'] = df['education'].str.lower()

#print(df['family_status'].unique())

#print(df['family_status_id'].unique())

#print(df.groupby('family_status')['family_status_id'].value_counts())

#print(df['gender'].unique())
df['gender'] = df['gender'].replace('XNA', 'M') # предположитеьно мужчина
#print(df['income_type'].unique())
#print(df['debt'].unique())

#обработка пропущеных значений в столбце доход

def new_total_income_count(row):
    income_type = row['income_type']
    total_income = row['total_income']
    mean_income = df[df['income_type'] == income_type][df['total_income'] != 0]['total_income'].mean()
    if total_income > 0:
        return  total_income
    else:
        return mean_income

df['new_total_income'] = df.apply(new_total_income_count, axis=1)

#обработка пропущеных значений в столбце стаж

def new_days_employed_count(row):
    employed = row['days_employed']
    age = row['dob_years']
    new_days_employed = (age - 18)*365
    
    if employed > 0:
        return  employed
    else:
        return new_days_employed

df['new_days_employed'] = df.apply(new_days_employed_count, axis=1)

54


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


### Вывод

В таблице найдено 54 дибликата, что составляет менее 1% данных, поэтому удаляем эти строки.
В столбце "дети" были обработаны артефакты.

Пустые столбцы возраст были обработаны с учетом трудового стажа. Оставшиеся 10 (с нулевым возрастом и трудовым стажем) строк будут обработы после создания категорий.

Соотношение образования с категорией:
education            
education_id

высшее               0                
начальное            3                 
неоконченное высшее  2                
среднее              1               
ученая степень       4


family_status          family_status_id

в разводе              3                    
вдовец / вдова         2                     
гражданский брак       1                    
женат / замужем        0                   
не женат / не замужем  4     


Единственная запись XNA - возраст 24 года/трудовой стаж 6.5 лет/2-е детей, предположитеьно мужчина.

Данные в остальных столбцах в пределах нормы.

Пропущеные значения в столбце "ежемесяцный доход" были обработаны в соответствии с категориями зантости.

Пропущенные значения в столбце трудовой стаж были сопоставлены с возрастом и отражены в таблице.

### Лемматизация

In [74]:
#print(df['purpose'].unique())

def purpose_lemmas(row):
    text = row['purpose']
    lemmas = ' '.join(m.lemmatize(text))
    return lemmas
    
df['purpose_after_lemmas'] = df.apply(purpose_lemmas, axis =1)

#print(df['purpose_after_lemmas'].unique())

### Вывод

Для удобства категоризации целей получения кредита была произведена лемматизация данных в столбце "цели".

Основные категории:

Приобретение/действия с жильем/недвижимостью 
Покупка автомобиля 
Получение образования
Оплата свадебного торжества

### Категоризация данных

In [75]:
#функция для сортировки целей кредитования по категориям
def purpose_to_category(row):
    purpose = row['purpose_after_lemmas']
    if 'жилье' in purpose:
        category = 1
    elif 'недвижимость' in purpose:
        category = 1
    elif 'автомобиль' in purpose:
        category = 2
    elif 'образование' in purpose:
        category = 3
    elif 'свадьба' in purpose:
        category = 4
    else:
        category = 0
    return category
    
df['purpose_category'] = df.apply(purpose_to_category, axis =1)

#разбивка доходов по категориям 
df['new_total_income'] = df['new_total_income'].astype(int)

import numpy as np
sum_income = df['new_total_income']
a = np.array(sum_income)
category1 = np.percentile(a, 13.4)
category2 = np.percentile(a, 41.2)
category3 = np.percentile(a, 80)
category4 = np.percentile(a, 90.9)
category5 = np.percentile(a, 98.2)
category6 = np.percentile(a, 99.3)

def income_to_category(row):
    
    income = row['new_total_income']
   
    if income < category1:
        income_category = 'крайне нищие'
    elif category1 <= income < category2:
        income_category = 'нищие'
    elif category2 <= income < category3:
        income_category = 'бедные'
    elif category3 <= income < category4:
        income_category = 'ниже среднего'
    elif category4 <= income < category5:
        income_category = 'средний достаток'
    elif category5 <= income < category6:
        income_category = 'состоятельные'
    elif income >= category6:
        income_category = 'богатые'
    else:
        income_category = 'none'
    return income_category

    
df['income_category'] = df.apply(income_to_category, axis =1)

### Вывод

Категориям целей крдитования были присвоены индексы для удобства отображения. 

Информация о доходах была категоризирована в семи категориях (данные соотнесены с процентным соством общества по доходам в РФ за 2018 год).

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [76]:
df.groupby('children')['debt'].agg(['count','mean'])

,count,mean
children,,
0,14107,0.075353
1,4856,0.091639
2,2128,0.094925
3,330,0.081818
4,41,0.097561
5,9,0.000000


### Вывод

Наличие детей не влияет на возврат кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [77]:
display(df.groupby('family_status')['debt'].agg(['count','mean']))
df.pivot_table(index='family_status', columns ='children',values=['debt'],aggfunc=['mean','count'])

,count,mean
family_status,,
Не женат / не замужем,2810,0.097509
в разводе,1195,0.071130
вдовец / вдова,959,0.065693
гражданский брак,4163,0.093202
женат / замужем,12344,0.075421


mean                                               \
                           debt                                                
children                      0         1         2         3         4    5   
family_status                                                                  
Не женат / не замужем  0.092838  0.114537  0.119048  0.125000  0.500000  NaN   
в разводе              0.070153  0.066456  0.096386  0.090909  0.000000  NaN   
вдовец / вдова         0.062574  0.086420  0.125000  0.000000  0.000000  NaN   
гражданский брак       0.083546  0.117882  0.092958  0.142857  0.000000  0.0   
женат / замужем        0.069049  0.082224  0.093552  0.068273  0.103448  0.0   

                        count                                    
                         debt                                    
children                    0       1       2      3     4    5  
family_status                                                    
Не женат / не замужем  2262.0   454.0    84.0    8.0   2.0  NaN  
в разводе               784.0   316.0    83.0   11.0   1.0  NaN  
вдовец / вдова          847.0    81.0    24.0    6.0   1.0  NaN  
гражданский брак       2741.0  1001.0   355.0   56.0   8.0  2.0  
женат / замужем        7473.0  3004.0  1582.0  249.0  29.0  7.0

### Вывод

Вдовцы/вдовы чаще возвращают кредит в срок, чем остальные. Не женатые/ не замужные реже всех возвращают кредиты в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [78]:
df.groupby('income_category')['debt'].agg(['count','mean']).sort_values('mean',ascending=False)

,count,mean
income_category,,
бедные,8407,0.085643
нищие,5892,0.085200
крайне нищие,2877,0.076121
средний достаток,1567,0.073389
ниже среднего,2341,0.069628
богатые,151,0.059603
состоятельные,236,0.055085


### Вывод

Из полученной таблицы можно сделать вывод, что состоятельные люди чаще всех возвращают кредиты в срок. Три категории людей с наименьшими доходами возглавляют рейтинг неплатильщиков.

- Как разные цели кредита влияют на его возврат в срок?

In [79]:
df.groupby('purpose_category')['debt'].agg(['count','mean'])

,count,mean
purpose_category,,
1,10814,0.072314
2,4308,0.093547
3,4014,0.092177
4,2335,0.079657


### Вывод

Кредиты которые берутся под покупку/действия с недвижимостью чаще возвращают в срок. Кредиты на покупку автомобиля и образование реже всего.

### Шаг 4. Общий вывод

Полученные данные были проанализированны, пропущенные значения были восстановлены на основе имеющейся ифнормации. В рамках анализа были выделенны категории доходов клиентов, а так же систематизированы цели на которые клиенты планировали потратить денежные средства.
В целом, уровень возвращаемости кредита составляет от 6% до 10%.
Наличие детей не влияет на на возврат кредита в срок (средние значнения в предлах 7.5% - 9.7%), в отличии семейного статуса - не женатые/ не замужные реже всех возвращают долги в срок(9.7%). Чаще всех вовремя плятят по кредитам вдовы/вдовцы(только 6.5% просрочек) 
Объем источников дохода так же влияет на возврат кредита в срок- клиенты с наименьшим уровнем дохода склонны реже возвращать кредиты вовремя. Две наиболее крупные категории имеют средние значение в 8.5%, в то время как состоятельные люди не платят вовремя только в минимальных 5.5% случаев.
Кредиты направленые на покупку недвижимости возвращают в срок чаще остальных (7.2%), реже вовремя платят по кредитам за образование/автокредиты (около 9.3% случаев).